In [38]:
import logging
import sys
import nest_asyncio

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

nest_asyncio.apply()

In [39]:
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    Response,
    
)
import pinecone
import openai
import streamlit as st
from llama_index.vector_stores import PineconeVectorStore
from llama_index.llms import OpenAI
from llama_index.evaluation import SemanticSimilarityEvaluator

pd.set_option("display.max_colwidth", 0)

In [40]:
openai.api_key = st.secrets.openai_key
api_key = "2bcecfc8-35f9-44dc-9459-ab0cc2781b29"
pinecone.init(api_key=api_key, environment="gcp-starter")
pinecone_index = pinecone.Index("tailwind-ada")

In [41]:
pinecone_index = pinecone.Index("tailwind-ada")
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
service_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo", temperature=0.5, system_prompt="You are an expert on TailwindCSS and your job is to answer technical questions. Assume that all questions are related to TailwindCSS. Keep your answers technical and based on facts – do not hallucinate features."))
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, service_context=service_context)

In [42]:
evaluator = SemanticSimilarityEvaluator()

In [44]:
chat_engine = index.as_chat_engine(chat_mode="condense_question", verbose=True)
query = "What is border-radius? And how do I use it in TailwindCSS?"
response = chat_engine.chat(query)
reference = "It is a property that allows you to round the corners of an element. You can use it in TailwindCSS by adding the rounded class to an element. For example, rounded-lg will round the corners of an element by 0.5rem."

In [45]:
result = await evaluator.aevaluate(
    response=response.response,
    reference=reference,
)
print("Score: ", result.score)
print("Passing: ", result.passing)  # default similarity threshold is 0.8

,Query,Response,Source,Evaluation Result
0,What is border-radius? And how do I use it in TailwindCSS?,"Border-radius is a CSS property that allows you to round the corners of an element. In Tailwind CSS, you can use the border-radius utility classes to easily apply different border radius values to your elements. To use the border-radius utility classes in Tailwind CSS, you can add the `rounded` class to an element to apply a default border radius. Tailwind CSS also provides additional utility classes for different border radius sizes, such as `rounded-sm`, `rounded-md`, `rounded-lg`, and `rounded-full`. If you want to customize the default border radius values in Tailwind CSS, you can edit the `theme.borderRadius` section of your Tailwind config file. By default, Tailwind provides five border radius size utilities, but you can change, add, or remove these by modifying the values in the config file. For example, if you want to change the default border radius value to `4px`, you can add the following code to your Tailwind config file: ```javascript module.exports = { theme: { borderRadius: { DEFAULT: '4px', }, }, }; ``` This will update the default border radius value used by the `rounded` class. Additionally, you can also use arbitrary values for the border radius by using the `rounded-[value]` class. For example, if you want to apply a border radius of `12px`, you can use the `rounded-[12px]` class. Remember to refer to the Tailwind CSS documentation for more information on customizing the default theme and using the border-radius utility classes.","--- ## Applying conditionally ### Hover, focus, and other states ### Breakpoints and media queries --- ## Using custom values ### Customizing your theme By default, Tailwind provides five border radius size utilities. You can change, add, or remove these by editing the `theme.borderRadius` section of your Tailwind config. ```diff-js {{ filename: 'tailwind.config.js' }} module.exports = { theme: { borderRadius: { 'none': '0', - 'sm': '0.125rem', - DEFAULT: '0.25rem', + DEFAULT: '4px', - 'md': '0.375rem', - 'lg': '0.5rem', - 'full': '9999px', + 'large': '12px', } } } ``` Learn more about customizing the default theme in the [theme customization](/docs/theme#customizing-t...",Pass
